In [1]:
import os
import sys
from pathlib import Path
DIR_PROYECTO = Path(os.getcwd()).parent.resolve()
sys.path.append( str(DIR_PROYECTO) )

import lib
import numpy  as np
from lib.energia import curva_de_potencia

In [4]:
def energia_datos(ws):
    ws = np.copy(ws)
    potencia = curva_de_potencia(ws)
    energia = np.sum(potencia)
    return energia  / 1_000

def energia_con_prom_suma(ws,remover):
    ws = np.copy(ws)

    # Corrección exclusión valores menores a 3.5
    if remover:
        idx = ws > 3.5 
        ws  = ws[idx]
    
    n = ws.size

    ws_prom  = np.mean(ws)
    ws_prom_r  = np.repeat( np.mean(ws_prom), n)
    potencia = curva_de_potencia(ws_prom_r)
    energia  = np.sum(potencia)
    return energia / 1_000, ws_prom


def MAPE(v_r, v_p):
    return 100*np.mean(np.abs((v_r-v_p)/v_r))

In [5]:
def energia_anual(hasta_año):

    # Referencia
    años  = list(range(2001,hasta_año+1))
    ws    = lib.cargar_datos(año=años)
    _, _, k, lambda_o = lib.ajustar_weibull(ws)
    energia_anual = 365*24*lib.calcular_energia(ws) / ws.size

    # Evaluación
    ws_e = lib.cargar_datos(año=hasta_año+1)
    energia_e   = energia_datos(ws_e)
    
    # Modo de promedio quitando menores a 3.5
    energia_prom_suma_c , vel_prom_c = energia_con_prom_suma(ws, False)
    energia_prom_suma_s , vel_prom_s = energia_con_prom_suma(ws, True ) 
    energia_prom_suma_c = energia_prom_suma_c / (1 + hasta_año-2001)
    energia_prom_suma_s = energia_prom_suma_s / (1 + hasta_año-2001)


    # Modo de promedio con valores de 3.5

    # MAPE
    print("     {:4.0f}: {:7.2f} MWh".format(hasta_año+1, energia_e,))
    print("2001-{:4.0f}: {:7.2f} MWh | k={:5.3f} l={:5.3f} | MAPE={:5.2f}%".format(hasta_año, energia_anual,k,lambda_o,MAPE(energia_e, energia_anual)))
    print("2001-{:4.0f}: {:7.2f} MWh  (con vel prom > 3.5)| vel prom={:4.2f}    | MAPE={:5.2f}%".format(hasta_año, energia_prom_suma_c, vel_prom_c, MAPE(energia_e, energia_prom_suma_c)))
    print("2001-{:4.0f}: {:7.2f} MWh  (con vel prom todas)| vel prom={:4.2f}    | MAPE={:5.2f}%".format(hasta_año, energia_prom_suma_s, vel_prom_s, MAPE(energia_e, energia_prom_suma_s)))



print("---------------")
print("Energía anuales")
print("---------------")
print(" * Weibull formada con los datos del periodo")
print(" * Con la weibull se calcula la energía anual")
print(" * Mape de la energía anual del año de evaluación vz energía anual con weibull del periodo\n")

for año in range(2017,2022):
    energia_anual(año)
    print("-"*60)

---------------
Energía anuales
---------------
 * Weibull formada con los datos del periodo
 * Con la weibull se calcula la energía anual
 * Mape de la energía anual del año de evaluación vz energía anual con weibull del periodo

     2018: 9134.20 MWh
2001-2017: 9371.37 MWh | k=2.610 l=7.546 | MAPE= 2.60%
2001-2017: 7386.69 MWh  (con vel prom > 3.5)| vel prom=6.71    | MAPE=19.13%
2001-2017: 8549.33 MWh  (con vel prom todas)| vel prom=7.30    | MAPE= 6.40%
------------------------------------------------------------
     2019: 9559.52 MWh
2001-2018: 9360.23 MWh | k=2.600 l=7.541 | MAPE= 2.08%
2001-2018: 7366.54 MWh  (con vel prom > 3.5)| vel prom=6.71    | MAPE=22.94%
2001-2018: 8542.71 MWh  (con vel prom todas)| vel prom=7.30    | MAPE=10.64%
------------------------------------------------------------
     2020: 9700.29 MWh
2001-2019: 9383.62 MWh | k=2.596 l=7.549 | MAPE= 3.26%
2001-2019: 7392.98 MWh  (con vel prom > 3.5)| vel prom=6.72    | MAPE=23.79%
2001-2019: 8565.06 MWh  (con